# Modelo de clasificación binaria para predecir si el contrato de un jugador debería ser renovado o no.

#### Usaremos la librería SciKitLearn para implementar una técnica de machine learning llamada RandomForestClassifier, la cual combina resultados de varios árboles de decisión para formar una predicción final. Obtendremos de la predicción el valor de una variable binaria "y", que puede ser 1 (se recomienda renovar) o 0 (No se recomienda renovar).

In [1]:
# Instalacion de SciKit-Learn
#!pip install scikit-learn

In [2]:
#Importamos librerías necesarias y modelos de machine learning de SciKitLearn 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [3]:
# Cargar la tabla de los jugadores de los Lakers (21 a 24)
df_players_lakers = pd.read_csv(r"C:\Users\usuario\Analisis_de_estadisticas_de_rendimiento_de_jugadores_NBA\ETL\lakers_21_24.csv")

In [4]:
# Ocultamos advertencias innecesarias
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Cargamos los datos del dataframe 
df = df_players_lakers.copy()

# Definimos la variable objetivo "y" (1 = Renovar, 0 = No renovar) según criterios de renovación típicamente considerados en NBA para unas pocas variables clave de rendimiento como fg%, ast y pts prom. por partido
y = ((df["PTS"] >= 10) & (df["AST"] >= 5) & (df["FG%"] >= 0.45)).astype(int)

# Definimos las estadísticas a considerar por el modelo en la variable predictora "x" (todas las estadísticas de las que el modelo aprenderá para predecir las estadísticas clave de renovación "y")
estadisticas_rendimiento_completas = ["Age", "G", "GS", "MP", "FG", "FGA", "FG%", "3P", "3PA", "3P%", "2P", "2PA", "2P%", 
"FT", "FTA", "FT%", "ORB", "DRB", "TRB", "PTS", "AST", "STL", "BLK", "TOV", "PF", "eFG%"]

# Creamos dataframe de "x" con todas las estadisticas de rendimiento del punto anterior
X = df[estadisticas_rendimiento_completas]

# Dividimos los datos del dataframe en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalamos los datos para mejorar el desempeño del modelo (usamos StandardScaler)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenamos el modelo de clasificación RandomForest
model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced') # n_estimators=100 significa que el modelo está utilizando 100 árboles de predicción
model.fit(X_train_scaled, y_train)

# Probamos el modelo. Hace predicciones a partir del 80% de los datos de entrenamiento (los que conocía) y las compara con el 20% de los datos de prueba (los que no conocía)
y_pred = model.predict(X_test_scaled)

# Evaluamos la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy:.4f}")
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))

Precisión del modelo: 0.9474

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        17
           1       1.00      0.50      0.67         2

    accuracy                           0.95        19
   macro avg       0.97      0.75      0.82        19
weighted avg       0.95      0.95      0.94        19



Vemos que la precision es alta tanto para 0 como para 1, pero recall (sesnibilidad o exhaustividad) es baja para 1 (renovar), significa que el modelo predice pocos jugadores para renovar (prefiere evitar falsos positivos).

Si accuracy está por encima de 0.75 - 0.80, suele ser un modelo bastante bueno. Es el caso de nuestro modelo (Accuracy = 0.95).

### Prueba del modelo con 6 jugadores (TOP 3 mejores y peores). Evalúa estadísticas promedio de los jugadores para las temporadas de interés (las 3 últimas del dataframe).

In [6]:
# Listamos los jugadores a evaluar
jugadores_a_evaluar = ["Anthony Davis", "Austin Reaves", "LeBron James", "Isaiah Thomas", "Mac McClung", "Wenyen Gabriel"]

# Filtramos el DataFrame con los jugadores seleccionados
df_jugadores = df_players_lakers[df_players_lakers["Player"].isin(jugadores_a_evaluar)].copy()

# Agrupamos por jugador y calculamos el promedio de las estadísticas para las ultimas 3 temporadas (que son todas las del df)
df_promedio_jugadores_3_temp = df_players_lakers.groupby("Player")[X.columns].mean()

# Filtrar solo los jugadores que queremos evaluar
df_jugadores = df_promedio_jugadores_3_temp.loc[jugadores_a_evaluar].copy()

# Realizar la predicción usando el modelo entrenado
predicciones = model.predict(df_jugadores)

# Agregar la predicción al DataFrame
df_jugadores["Renovar"] = predicciones

# Mostrar los resultados
df_jugadores[["Renovar"]]

,Renovar
Player,
Anthony Davis,1
Austin Reaves,1
LeBron James,1
Isaiah Thomas,0
Mac McClung,0
Wenyen Gabriel,0


En la tabla que nos devolvió la aplicación del modelo de clasificación, vemos que de los jugadores que evaluamos, los jugadores clasificados con 1 (se recomienda renovar contrato) coinciden con el top 3 mejores del dashboard, y los clasificados con 0 (No se recomienda renovar contrato) coinciden con el top 3 peores del dashboard.

### Prueba del modelo para clasificar todo el plantel actual (temporada 23-24) en base a su rendimiento durante las últimas 3 temporadas. Se decide si se recomienda o no la renovación del contrato del jugador.

In [7]:
# Filtramos los jugadores de la temporada 2023-2024
df_2023_2024 = df_players_lakers[df_players_lakers['season_year'] == '2023-2024']

# Agrupamos por jugador del plantel actual y calculamos el promedio de las estadísticas
df_plantel_actual = df_2023_2024.groupby("Player")[X.columns].mean()

# Realizamos la predicción usando el modelo entrenado
predicciones = model.predict(df_plantel_actual)

# Agregamos la predicción al DataFrame (nueva columna llamada "Renovar")
df_plantel_actual["Renovar"] = predicciones

# Dividimos los jugadores a los que se recomienda renovar contrato (Renovar == 1) y a los que No (Renovar == 0)
jugadores_a_renovar = df_plantel_actual[df_plantel_actual["Renovar"] == 1]
jugadores_no_renovar = df_plantel_actual[df_plantel_actual["Renovar"] == 0]

# Mostramos los jugadores recomendados para renovar contrato
print("Jugadores recomendados para renovar contrato:")
print(jugadores_a_renovar[['Renovar']])

print("-----------------") 

# Mostramos los jugadores a los que no se recomienda renovar contrato
print("Jugadores NO recomendados para renovar contrato:")
print(jugadores_no_renovar[['Renovar']])

Jugadores recomendados para renovar contrato:
                  Renovar
Player                   
Anthony Davis           1
Austin Reaves           1
D'Angelo Russell        1
LeBron James            1
-----------------
Jugadores NO recomendados para renovar contrato:
                     Renovar
Player                      
Alex Fudge                 0
Cam Reddish                0
Christian Wood             0
Colin Castleton            0
D'Moi Hodge                0
Dylan Windler              0
Gabe Vincent               0
Harry Giles                0
Jalen Hood-Schifino        0
Jarred Vanderbilt          0
Jaxson Hayes               0
Max Christie               0
Maxwell Lewis              0
Rui Hachimura              0
Skylar Mays                0
Spencer Dinwiddie          0
Taurean Prince             0


# Repetimos el modelo pero ahora basando los criterios de clasificación en estadísticas claves en defensa

In [8]:
# Cargamos los datos del dataframe 
df = df_players_lakers.copy()

# Definimos la variable objetivo "y" (1 = Renovar, 0 = No renovar) según criterios de renovación típicamente considerados en NBA para unas pocas variables clave de rendimiento defensivo como drb, stl, blk y orb prom. por partido
y = ((df["DRB"] >= 5) & (df["STL"] >= 1) & (df["BLK"] >= 2 & (df["ORB"] >= 2))).astype(int)

# Definimos las estadísticas a considerar por el modelo en la variable predictora "x" (todas las estadísticas de las que el modelo aprenderá para predecir las estadísticas clave de renovación "y")
estadisticas_rendimiento_completas = ["Age", "G", "GS", "MP", "FG", "FGA", "FG%", "3P", "3PA", "3P%", "2P", "2PA", "2P%", 
"FT", "FTA", "FT%", "ORB", "DRB", "TRB", "PTS", "AST", "STL", "BLK", "TOV", "PF", "eFG%"]

# Creamos dataframe de "x" con todas las estadisticas de rendimiento del punto anterior
X = df[estadisticas_rendimiento_completas]

# Dividimos los datos del dataframe en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalamos los datos para mejorar el desempeño del modelo (usamos StandardScaler)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenamos el modelo de clasificación RandomForest
model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced') # n_estimators=100 significa que el modelo está utilizando 100 árboles de predicción
model.fit(X_train_scaled, y_train)

# Probamos el modelo (hacemos las predicciones)
y_pred_def = model.predict(X_test_scaled)

# Evaluamos la precisión del modelo 
accuracy = accuracy_score(y_test, y_pred_def)
print(f"Precisión del modelo: {accuracy:.4f}")
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_def))

Precisión del modelo: 1.0000

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       1.00      1.00      1.00         1

    accuracy                           1.00        19
   macro avg       1.00      1.00      1.00        19
weighted avg       1.00      1.00      1.00        19



### Prueba del modelo para clasificar todo el plantel actual (temporada 23-24) en base a su rendimiento según criterios DEFENSIVOS durante las últimas 3 temporadas. Se decide si se recomienda o no la renovación del contrato del jugador.

In [9]:
# Filtramos los jugadores de la temporada 2023-2024
df_2023_2024 = df_players_lakers[df_players_lakers['season_year'] == '2023-2024']

# Agrupamos por jugador del plantel actual y calculamos el promedio de las estadísticas
df_plantel_actual = df_2023_2024.groupby("Player")[X.columns].mean()

# Realizamos la predicción usando el modelo entrenado
predicciones = model.predict(df_plantel_actual)

# Agregamos la predicción al DataFrame (nueva columna llamada "Renovar")
df_plantel_actual["Renovar"] = predicciones

# Dividimos los jugadores a los que se recomienda renovar contrato (Renovar == 1) y a los que No (Renovar == 0)
jugadores_a_renovar = df_plantel_actual[df_plantel_actual["Renovar"] == 1]
jugadores_no_renovar = df_plantel_actual[df_plantel_actual["Renovar"] == 0]

# Mostramos los jugadores recomendados para renovar contrato
print("Jugadores recomendados para renovar contrato según criterios defensivos:")
print(jugadores_a_renovar[['Renovar']])

print("-----------------") 

# Mostramos los jugadores a los que no se recomienda renovar contrato
print("Jugadores NO recomendados para renovar contrato según criterios defensivos:")
print(jugadores_no_renovar[['Renovar']])

Jugadores recomendados para renovar contrato según criterios defensivos:
                   Renovar
Player                    
Anthony Davis            1
Austin Reaves            1
Cam Reddish              1
Christian Wood           1
D'Angelo Russell         1
Jarred Vanderbilt        1
LeBron James             1
Max Christie             1
Rui Hachimura            1
Spencer Dinwiddie        1
Taurean Prince           1
-----------------
Jugadores NO recomendados para renovar contrato según criterios defensivos:
                     Renovar
Player                      
Alex Fudge                 0
Colin Castleton            0
D'Moi Hodge                0
Dylan Windler              0
Gabe Vincent               0
Harry Giles                0
Jalen Hood-Schifino        0
Jaxson Hayes               0
Maxwell Lewis              0
Skylar Mays                0


# Conclusiones

#### - Según el modelo de clasificación basado en estadísticas principalmente ofensivas, se recomienda renovar contrato solo a 4 jugadores de 22 del plantel actual.

#### - Según el modelo de clasificación basado en estadísticas defensivas, se recomienda renovar contrato a 11 jugadores de 22 del plantel actual, pero en estos 11 se incluyen los mismos 4 recomendados por su rendimiento ofensivo. 

#### - Si tenemos en cuenta ambos grupos de criterios del modelo de clasificación se recomienda renovar contrato a 11 jugadores de un total de 22 del plantel actual, por lo que la mitad del plantel podría mantenerse.

#### - Lista de jugadores a los que se recomienda renovar contrato:

Anthony Davis            
Austin Reaves            
Cam Reddish              
Christian Wood           
D'Angelo Russell         
Jarred Vanderbilt        
LeBron James             
Max Christie             
Rui Hachimura            
Spencer Dinwiddie        
Taurean Prince   